In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
sns.set()

pd.set_option("display.max_rows", 5)
pd.set_option('display.max_columns',100)

In [38]:
df_form1 = pd.read_excel('dataset/formulario1.xlsx')
df_form1['bad_smell'] = df_form1['bad_smell'].str.lower().replace({"long method": "Long Method",
                           "métodos longos": "Long Method",
                           "método grande": "Long Method",
                           "longo método": "Long Method",
                           "método longo": "Long Method",
                           "método longo": "Long Method",
                           "método longo, muitos comentários":"Long Method",
                           "duplicade code":"Duplicade Code",
                           "código duplicado":"Duplicade Code",
                           "duplicated code":"Duplicade Code",
                           "codigo duplicado":"Duplicade Code",
                           "código duplicado,":"Duplicade Code",
                           "código duplicado + outro":"Duplicade Code",
                           "complexidade ciclomática": "Complexidade Ciclomática",
                           "método longo com grande complexidade ciclomatica":"Complexidade Ciclomática",
                           "método longo com grande complexidade ciclomatica, método invejoso":"Complexidade Ciclomática",
                           "grande complexidade ciclomática":"Complexidade Ciclomática",
                           "relativa complexidade ciclomática":"Complexidade Ciclomática",
                           "classe grande, muitos comentários": "Long Class",
                           "classe grande":"Long Class",
                           "classe longa":"Long Class",
                           "long class":"Long Class",
                           "god class":"Long Class",
                           "god method":"Long Method",
                           "longa lista de parâmetros":"Long Parameter List",
                           "métodos com muitos parâmetros":"Long Parameter List",
                           "método com muitos parâmetros":"Long Parameter List",
                           "long parameter list":"Long Parameter List",
                           "longa lista de parametros":"Long Parameter List",
                           "muitos parâmetros":"Long Parameter List",
                           "classe de dados":"Data Class",
                           "switch":"Switch",
                           "switch case":"Switch",
                           "uso indevido de switch":"Switch",
                           "switch statement":"Switch",
                           "método invejoso":"Envy",
                           "envy class":"Envy",
                           "sem valor": "No Value",
                           "nome do método pode ser melhor": "No Value",
                           "não utilização de padrão de nomenclatura": "No Value",
                           "se algo no cliente for alterado + precisará alterar em diversos arquivos + códigos semelhantes.": "No Value"})
display(df_form1['bad_smell'].unique())


df_form1['classe'] = df_form1['classe'].replace({
    "AGENCIA":"Agencia",
    "Agência":"Agencia",
    "agencia":"Agencia",
    "EMPRESA":"Empresa",
    "carteira":"Carteira"
})
df_form1['metodo'] = df_form1['metodo'].str.lower()
df_form1 = df_form1[df_form1['classe'] != 'sem valor']
df_form1 = df_form1[df_form1['bad_smell'] != 'No Value']
display(df_form1['classe'].unique())
display(df_form1)

array(['Long Method', 'Duplicade Code', 'Complexidade Ciclomática',
       'Long Class', 'No Value', 'Long Parameter List', 'Data Class',
       'Switch', 'Envy'], dtype=object)

array(['Agencia', 'Carteira', 'Empresa', 'Climas', 'Estacao', 'Cliente',
       'Destino'], dtype=object)

,nome,matricula,horario_inicio,horario_fim,classe,metodo,bad_smell,descricao,sugestao,refatoracao,nivel_dificuldade_encontrar,certeza_do_bad_smell
0,Thiago Abreu Lopes,469845.0,15:54:00,15:59:00,Agencia,agendaviagem,Long Method,Método com muitas linhas de código,Fracionar esse único módulo em módulos menores\n,"Sim, a longo prazo pode vir a ser um problema ...",Fácil,Muita Certeza
1,Thiago Abreu Lopes,469845.0,15:59:00,16:01:00,Agencia,agendaviagemparaestacao,Long Method,Método com muitas linhas de código,Fracionar esse único módulo em módulos menores,"Sim, a longo prazo pode vir a ser um problema ...",Intermediário,Intermediário
...,...,...,...,...,...,...,...,...,...,...,...,...
284,Vinicius Tabosa dos Santos,472107.0,16:14:00,16:19:00,Agencia,agendaviagemparaestado,Duplicade Code,Sem valor,Criar uma classe com seus próprios métodos que...,"Sim, o método é logo e não faz sentido ficar n...",Fácil,Intermediário
285,Vinicius Tabosa dos Santos,472107.0,16:14:00,16:19:00,Agencia,agendaviagem2,Duplicade Code,Sem valor,Criar uma classe com seus próprios métodos que...,"Sim, o método é logo e não faz sentido ficar n...",Fácil,Intermediário


In [39]:
df_form4 = pd.read_excel('dataset/horáculo.xlsx')
df_form4['metodo'] = df_form4['metodo'].str.lower()
df_form4

,classe,metodo,bad smell
0,Cliente,cliente,Duplicade Code
1,Cliente,cliente,Long Parameter List
...,...,...,...
40,Carteira,construtor,Long Parameter List
41,Carteira,carteira,Long Parameter List


In [40]:
horaculo = df_form4['metodo'].unique()
df_form1['metodo'] = df_form1['metodo'].replace({
    'agenciaviagem':'agendaviagem',
    'agendaviagemparaestaacao':'agendaviagemparaestacao',
    'compranodebitouptade':'compranodebitoupdate',
    'public agencia':'agencia',
    'calculasaldoagencia;':'calculasaldoagencia',
    'calculasaldoagencia2;': 'calculasaldoagencia2',
    'compranocredito;': 'compranocredito',
    'agenciacalculasaldoagencia3':'calculasaldoagencia3',
    'adiciona destido':'adicionadestido',
    'calculasaldoagencia1':'calculasaldoagencia',
    'agendaviagemparaestado':'agendaviagemparaestacao',
    'adicionadestido':'adicionadestino',
    'a classe toda':'sem valor',
    'construtores da classe':'empresa',
    'construtor da classe': 'cliente'
})

## Taxa de acurácia por classe

In [69]:
for classe in df_form1['classe'].unique():
    data = df_form1[df_form1['classe'] == classe]
    cont = 0
    for metodo in df_form1['metodo'].unique():
        data_metodo = data[data['metodo']==metodo]
        horaculo_for_method = list(df_form4[df_form4['metodo']==metodo]['bad smell'])
        bad_smells = list(data_metodo['bad_smell'])
        for bad in data_metodo['bad_smell']:
            if(bad in horaculo_for_method): cont +=1
    print(classe)
    print(cont/len(data)*100,end='\n\n')

Agencia
97.61904761904762

Carteira
98.4126984126984

Empresa
76.08695652173914

Climas
0.0

Estacao
0.0

Cliente
80.0

Destino
100.0



## Código para verificar onde ta errando

In [ ]:
for classe in df_form1['classe'].unique():
    data = df_form1[df_form1['classe'] == classe]
    cont = 0
    for metodo in df_form1['metodo'].unique():
        data_metodo = data[data['metodo']==metodo]
        horaculo_for_method = list(df_form4[df_form4['metodo']==metodo]['bad smell'])
        bad_smells = list(data_metodo['bad_smell'])
        for bad in data_metodo['bad_smell']:
            if(bad in horaculo_for_method): cont +=1
            else:
                print(f"Classe:{classe}, Metodo: {metodo}, Bad Smell: {bad}", end='\n\n')